In [2]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd

## Access the main page of lacentrale to get the list of car makes

In [20]:
def init_car_make_names(url):
    response=r.get(url).content
    soup=BeautifulSoup(response)
    tabs=soup.select('div.topBrands.clearTablet li')
    list_brands=[i.text.strip() for i in tabs][:-1]
    return list_brands
list_make = init_car_make_names('https://www.lacentrale.fr')

In [21]:
list_make

['Audi',
 'Bmw',
 'Citroen',
 'Dacia',
 'Fiat',
 'Ford',
 'Mercedes',
 'Nissan',
 'Opel',
 'Peugeot',
 'Renault',
 'Toyota',
 'Volkswagen']

# Access the page of a specified vehicule make
# Example, here the 'audi' make

In [22]:
def init_url_announces(make,num_page):
    # access to the page of a specified car make
    # example, here the 'audi'
    
    if(num_page==1):
        url_make=f'https://www.lacentrale.fr/occasion-voiture-marque-{make.lower()}.html?'
    else:
        url_make=f'https://www.lacentrale.fr/occasion-voiture-marque-{make.lower()}-{num_page}.html?'
    
    response=r.get(url_make).content
    soup=BeautifulSoup(response)
    tabs=soup.select('.adContainer>a')
    list_url_car=[tab.get('href') for tab in tabs]
    return list_url_car

list_url_car = init_url_announces('Audi',2)

In [23]:
list_url_car

['/auto-occasion-annonce-87101688627.html',
 '/auto-occasion-annonce-87101714896.html',
 '/auto-occasion-annonce-69106278665.html',
 '/auto-occasion-annonce-87101713561.html',
 '/auto-occasion-annonce-87101684279.html',
 '/auto-occasion-annonce-69106185653.html',
 '/auto-occasion-annonce-87101695792.html',
 '/auto-occasion-annonce-66101131434.html',
 '/auto-occasion-annonce-87101657625.html',
 '/auto-occasion-annonce-87101710990.html',
 '/auto-occasion-annonce-87101659004.html',
 '/auto-occasion-annonce-87101678633.html',
 '/auto-occasion-annonce-87101702022.html',
 '/auto-occasion-annonce-69106279195.html',
 '/auto-occasion-annonce-87101713054.html',
 '/auto-occasion-annonce-87101701718.html']

## Collect vehicle data

In [24]:
def collect_data_car(make,url_car_announce,it):

    print(f'url announce {url_car_announce}\n')
    resp=r.get(url_car_announce).content

    soup2=BeautifulSoup(resp)
    info={}
    info={'Marque':make}
    reference_bloc = soup2.select('.cbm-moduleInfos__RefAnnonce span')[0]
    ref_info = reference_bloc.text.split(' | ')

    #PROCESS TO DETERMINE THE REFERENCE OF ANNOUNCE
    #CASE SIMPLE REF
    if len(ref_info)==1:
        ref=ref_info[0].split(' : ')[1]
        reference_announce=ref_info[0]     
    # CASE COMPOUNED REF
    else:
        reference_announce = ref_info[1]
        ref = reference_announce.split(' : ')[1]       
    info.update({'Référence annonce':ref})
    
    vehicule_name = soup2.select('.cbm-moduleInfos__informationListFirst')[0].text
    info.update({'Nom':vehicule_name})  

    
    # GET COMMERCIAL NAME AND PRICE CAR FROM JSON DATA 
    resp2 = r.get(f'https://lc-user-classified-data.carboatservices.fr/v1/me/classifieds/hydrate?refs={ref}').json()
    json_data = resp2.get('data')[0]
    vehicle_commercialName=json_data.get('vehicle_commercialName')
    info.update({'Modèle':vehicle_commercialName})
    vehicule_price = json_data.get('price')
    info.update({'Prix':vehicule_price})

    
    
    # GET VEHICULE YEAR AND VEHICULE MILEAGE JSON DATA
    vehicle_year = json_data.get('vehicle_year')
    vehicle_mileage = json_data.get('vehicle_mileage')
    
    # GET GENERAL INFORMATION OF THE CAR
    information_lines = soup2.select('.cbm-moduleInfos__informationList li')
    list_information = [i.text.replace('?','') for i in information_lines]

    try:
        position=list_information.index("Éligible prime à la conversion : ")
        list_information[position]='Éligible prime à la conversion : Oui'
        list_information.remove('Suis-je éligible ')
    except ValueError:
        list_information.append('Éligible prime à la conversion : Non')
        
    #print(f'list information is :\n{list_information}\n')
    # BUILD A DICTIONARY INFO WITH GENERAL INFORMATION OF A VEHICLE 
    dict_info = {i.split(' : ')[0]:i.split(' : ')[1] for i in list_information}
    for i in dict_info:
        info.update({i:dict_info[i]})
    
    # UPDATE VEHICULE YEAR AND VEHICULE MILEAGE BY THOSE EXTRACT FROM JSON DATA
    info.update({'Kilométrage compteur km': vehicle_mileage})
    info.update({'Année': vehicle_year})
    # REMOVE INCORRECT VALUE OF MISE EN CIRCULATION
    del info['Mise en circulation']
    # REMOVE INCORRECT VALUE OF Kilométrage compteur
    del info['Kilométrage compteur']
    #print(f'info \n{info}\n')
    # BUILD A DF WITH ALL GENERAL INFORMATION OF A VEHICLE
    df=pd.DataFrame(info,index=[it])
    return df


df = collect_data_car('Audi','https://www.lacentrale.fr/auto-occasion-annonce-69106292215.html',1)
df

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106292215.html



,Marque,Référence annonce,Nom,Modèle,Prix,Année,Contrôle technique,Énergie,Boîte de vitesse,Couleur extérieure,Nombre de portes,Nombre de places,Première main (déclaratif),Puissance fiscale,Puissance din,Crit'Air,Émissions de CO2,Norme Euro,Éligible prime à la conversion,Kilométrage compteur km
1,Audi,E106292215,AUDI A3 III (2) SPORTBACK 1.6 TDI 110 BUSINESS...,A3 (3E GENERATION) SPORTBACK,14990,2016,non requis,Diesel,automatique,blanc,5,5,non,5 CV,110 ch,2,99 g/kmA,EURO6,Non,121500


## Scrap all anounces of one page

In [25]:
list_url_car

['/auto-occasion-annonce-87101688627.html',
 '/auto-occasion-annonce-87101714896.html',
 '/auto-occasion-annonce-69106278665.html',
 '/auto-occasion-annonce-87101713561.html',
 '/auto-occasion-annonce-87101684279.html',
 '/auto-occasion-annonce-69106185653.html',
 '/auto-occasion-annonce-87101695792.html',
 '/auto-occasion-annonce-66101131434.html',
 '/auto-occasion-annonce-87101657625.html',
 '/auto-occasion-annonce-87101710990.html',
 '/auto-occasion-annonce-87101659004.html',
 '/auto-occasion-annonce-87101678633.html',
 '/auto-occasion-annonce-87101702022.html',
 '/auto-occasion-annonce-69106279195.html',
 '/auto-occasion-annonce-87101713054.html',
 '/auto-occasion-annonce-87101701718.html']

In [26]:
url='https://www.lacentrale.fr'

In [37]:
import time
def scrap_anounces_car_make(make,list_url_car):
    dataframe = pd.DataFrame()
    i=0
    for url_car in list_url_car:
        i+=1
        hole_url_car=url+url_car
        dataframe = dataframe.append(collect_data_car(make,hole_url_car,i))
        time.sleep(1)  
    return dataframe

df = scrape_anounces_car_make('Audi',list_url_car)   


url announce https://www.lacentrale.fr/auto-occasion-annonce-87101688627.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101714896.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106278665.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101713561.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101684279.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106185653.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101695792.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-66101131434.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101657625.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101710990.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101659004.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101678633.html

url announce https://www.lacentrale.fr/auto-occasion

In [38]:
df.shape

(16, 24)

In [39]:
df

,Marque,Référence annonce,Nom,Modèle,Prix,Année,Contrôle technique,Énergie,Boîte de vitesse,Couleur extérieure,...,Première main (déclaratif),Puissance fiscale,Puissance din,Crit'Air,Émissions de CO2,Norme Euro,Éligible prime à la conversion,Kilométrage compteur km,Nombre de propriétaires,Garantie constructeur
1,Audi,W101688627,AUDI SQ5 (2) 3.0 V6 BITDI 340 PLUS QUATTRO TIP...,SQ5,47880,2016,non requis,Diesel,automatique,noir metal,...,non,23 CV,340 ch,2,177 g/kmE,EURO6,Non,42500,NaN,NaN
2,Audi,W101714896,AUDI SQ5 (2) 3.0 V6 BITDI 340 PLUS QUATTRO TIP...,SQ5,47800,2016,non requis,Diesel,automatique,gris foncé metal,...,non,23 CV,340 ch,2,177 g/kmE,EURO6,Non,49500,NaN,NaN
3,Audi,E106278665,AUDI A3 III SPORTBACK 1.4 TFSI 125 AMBIENTE S ...,A3 (3E GENERATION) SPORTBACK,14990,2014,requis,Essence,automatique,noir,...,non,7 CV,125 ch,1,113 g/kmB,EURO6,Oui,82500,NaN,NaN
4,Audi,W101713561,AUDI Q5 (2) 2.0 TDI 150 AMBIENTE,Q5,16900,2013,requis,Diesel,mécanique,blanc verni,...,non,8 CV,150 ch,2,139 g/kmC,EURO5,Non,120000,3,NaN
5,Audi,W101684279,AUDI A8 III 4.2 TDI 350 DPF AVUS QUATTRO TIPTR...,A8 (3E GENERATION),24990,2010,requis,Diesel,automatique,gris metal,...,non,24 CV,350 ch,2,195 g/kmE,EURO5,Non,132200,NaN,NaN
6,Audi,E106185653,AUDI A6 2.0 TDI 190CH DESIGN LUXE S S TRONIC U...,A6 (4E GENERATION),24900,2016,non requis,Diesel,automatique,noir,...,oui,10 CV,0 ch,NaN,110 g/kmB,NaN,Non,77800,NaN,NaN
7,Audi,W101695792,AUDI Q7 II V6 3.0 TFSI 333 QUATTRO TIPTRONIC 7PL,Q7 (2E GENERATION),55000,2016,non requis,Essence,automatique,blanc metal,...,oui,22 CV,333 ch,1,183 g/kmE,EURO6,Non,20,NaN,NaN
8,Audi,B101131434,AUDI A6 ALLROAD IV 3.0 V6 TDI 245 AMBITION LUX...,A6 (4E GENERATION) ALLROAD,22500,2014,requis,Diesel,automatique,noir metal,...,non,15 CV,245 ch,2,165 g/kmE,EURO5,Non,110900,2,NaN
9,Audi,W101657625,AUDI Q7 (2) 6.0 V12 TDI 500 6PL,Q7,37990,2011,requis,Diesel,automatique,bleu metal,...,non,41 CV,501 ch,2,298 g/kmG,EURO5,Non,109000,2,NaN
10,Audi,W101710990,AUDI A4 II TDI 130 PACK,A4 (2E GENERATION),2795,2003,requis,Diesel,mécanique,gris anthracite metal,...,non,7 CV,130 ch,4,NaN,EURO3,Non,216000,NaN,NaN


## Scraps first 50 pages 

In [40]:
from datetime import datetime    
        
        
if __name__=='__main__':
    df2 = pd.DataFrame()
    list_car_makes = init_car_make_names('https://www.lacentrale.fr')
    # display begin time
    print("Begin time =", datetime.now())
    # iterate through all car makes
    for make in list_car_makes:
        # scrap 2 recent pages of announces  
         for i in range(2):
            list_url_car=init_url_announces(make,i+1)
            df2 = df2.append(scrap_anounces_car_make(make,list_url_car))
            print(make)
            print(i+1)
     # display end time
    print("End time =", datetime.now())

Begin time = 2020-04-13 17:37:57.590203
url announce https://www.lacentrale.fr/auto-occasion-annonce-69106042397.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101651725.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101718784.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106202231.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106278661.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101711349.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106113243.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69105862968.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106256873.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69105999639.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106031176.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106292215.html

url announce

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101669123.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69105438684.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106321746.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106040712.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106335714.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106216580.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106266255.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106202632.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106220715.html

Dacia
1
url announce https://www.lacentrale.fr/auto-occasion-annonce-69106250062.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106103306.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69105998285.html

url announce https://www.lacentrale.fr/auto-

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106306023.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106283933.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106278664.html

Mercedes
1
url announce https://www.lacentrale.fr/auto-occasion-annonce-87101711372.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101689772.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106127848.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69105724210.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106043353.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101718255.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101692445.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101691178.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106278640.html

url announce https://www.lacentrale.fr/au

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101707790.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69105700228.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106007216.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106111964.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106193052.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101697282.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106282663.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106084438.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101701479.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101707047.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106185501.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106020454.html

Peugeot
2
url announce https://www.lacentrale.fr/aut

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106295922.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101699277.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101687739.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101644932.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-87101669189.html

url announce https://www.lacentrale.fr/auto-occasion-annonce-69106206813.html

Volkswagen
2
End time = 2020-04-13 17:50:39.549330


In [41]:
df2.shape

(416, 26)

In [45]:
df2.Marque.unique()

array(['Audi', 'Bmw', 'Citroen', 'Dacia', 'Fiat', 'Ford', 'Mercedes',
       'Nissan', 'Opel', 'Peugeot', 'Renault', 'Toyota', 'Volkswagen'],
      dtype=object)

In [46]:
df2.to_csv('data/vehicles_lacentrale.csv')

In [47]:
df2['Boîte de vitesse'].unique()

array(['automatique', 'mécanique'], dtype=object)

In [48]:
make = input(f'Please choose your desired vehicle make from this list :\n{list_make}\n')
min_price = input(f'Please choose your min price :\n');
max_price = input(f'Please choose your max price :\n');
type_energy = input(f'Please choose your type of energy :\nDiesel, Essence, Electrique, Hybride\n');
type_gearbox = input(f'Please choose your type gearbox :\nautomatique, mécanique\n')

Please choose your desired vehicle make from this list :
['Audi', 'Bmw', 'Citroen', 'Dacia', 'Fiat', 'Ford', 'Mercedes', 'Nissan', 'Opel', 'Peugeot', 'Renault', 'Toyota', 'Volkswagen']
Bmw
Please choose your min price :
5000
Please choose your max price :
15000
Please choose your type of energy :
Diesel, Essence, Electrique, Hybride
Essence
Please choose your type gearbox :
automatique, mécanique
mécanique


In [49]:
make

'Bmw'

In [50]:
filtered = df2[df2.Marque==make]

In [51]:
filtered

,Marque,Référence annonce,Nom,Modèle,Prix,Année,Contrôle technique,Énergie,Boîte de vitesse,Couleur extérieure,...,Crit'Air,Émissions de CO2,Norme Euro,Éligible prime à la conversion,Kilométrage compteur km,Garantie,Nombre de propriétaires,Garantie constructeur,Vérifié & Garanti,Provenance
1,Bmw,W101710863,BMW SERIE 1 (F20) (2) 118I LOUNGE 5P,SERIE 1 F20 5 PORTES,17870,2017,non requis,Essence,mécanique,gris metal,...,1,137 g/kmC,EURO6,Oui,35930,12 mois,NaN,NaN,NaN,NaN
2,Bmw,W101689603,BMW SERIE 1 (F20) (2) 118D URBANCHIC 5P,SERIE 1 F20 5 PORTES,15980,2016,non requis,Diesel,mécanique,noir metal,...,2,109 g/kmB,EURO6,Non,68480,12 mois,NaN,NaN,NaN,NaN
3,Bmw,E105884512,BMW SERIE 4 (F36) GRAN COUPE 418D M SPORT BVA8,SERIE 4 F36 GRAN COUPE,26890,2017,non requis,Diesel,automatique,noir,...,2,116 g/kmB,EURO6,Non,57900,NaN,NaN,NaN,NaN,NaN
4,Bmw,E106278669,BMW SERIE 5 GT (F07) 530DA XDRIVE 258 M SPORT,SERIE 5 GT F07,22990,2014,requis,Diesel,automatique,gris,...,2,163 g/kmE,EURO6,Non,157500,NaN,NaN,NaN,NaN,NaN
5,Bmw,E105859544,BMW X1 (F48) SDRIVE18D M SPORT BVA8,X1 F48,27790,2017,non requis,Diesel,automatique,blanc,...,2,114 g/kmB,EURO6,Non,55990,NaN,NaN,NaN,NaN,NaN
6,Bmw,E106100042,BMW X1 (F48) XDRIVE25D M SPORT BVA8,X1 F48,29890,2016,non requis,Diesel,automatique,noir,...,2,133 g/kmC,EURO6,Non,89468,NaN,NaN,NaN,NaN,NaN
7,Bmw,E106182526,BMW X1 (E84) (2) XDRIVE25D M SPORT BVA8,X1 E84,21980,2015,non requis,Diesel,automatique,blanc,...,2,145 g/kmD,EURO5,Non,79900,NaN,NaN,NaN,NaN,NaN
8,Bmw,W101711983,BMW SERIE 1 (F20) 116I 136 LOUNGE 5P,SERIE 1 F20 5 PORTES,12490,2013,requis,Essence,mécanique,blanc verni,...,1,125 g/kmC,EURO6,Non,78090,6 mois,NaN,NaN,NaN,NaN
9,Bmw,W101630830,BMW I3 94 AH +EDITION LODGE,I3,23970,2017,non requis,Electrique,automatique,noir metal,...,0,0 g/kmA,EURO6,Oui,18560,12 mois,NaN,NaN,NaN,NaN
10,Bmw,E106085247,BMW X3 (F25) (2) XDRIVE30DA 258 M SPORT,X3 F25,31890,2015,non requis,Diesel,automatique,noir,...,2,156 g/kmD,EURO6,Non,88850,NaN,NaN,NaN,NaN,NaN


In [97]:
filtered.shape

(32, 26)

In [99]:
list_modele = filtered.Modèle.unique()
modele = input(f'Please choose your desired model from this list :\n{list_modele}\n')

Please choose your desired modele from this list :
['TRANSPORTER 5' 'T-ROC' 'GOLF 7' 'TIGUAN 2' 'GOLF 6' 'GOLF 6 R'
 'SCIROCCO 2' 'TIGUAN' 'AMAROK' 'SHARAN 2' None 'GOLF SPORTSVAN']
GOLF 7


In [100]:
modele

'GOLF 7'

In [101]:
filtered_modele = filtered[filtered.Modèle==modele]

In [102]:
filtered_modele

,Marque,Référence annonce,Nom,Modèle,Prix,Année,Contrôle technique,Énergie,Boîte de vitesse,Couleur extérieure,...,Émissions de CO2,Norme Euro,Éligible prime à la conversion,Kilométrage compteur km,Couleur intérieure,Garantie,Nombre de propriétaires,Garantie constructeur,Vérifié & Garanti,Provenance
4,Volkswagen,W101713798,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,12900,2015,non requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO6,Non,94000,tissu noir,6 mois,2,NaN,NaN,NaN
6,Volkswagen,W101713398,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10900,2013,requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO5,Non,134000,tissu noir,6 mois,3,NaN,NaN,NaN
8,Volkswagen,W101691329,VOLKSWAGEN GOLF VII 1.6 TDI 110 BLUEMOTION TEC...,GOLF 7,13490,2016,non requis,Diesel,mécanique,noir metal,...,99 g/kmA,EURO6,Non,75000,tissu gris,6 mois,1,NaN,NaN,NaN
13,Volkswagen,E106324636,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10990,2013,requis,Diesel,automatique,gris,...,102 g/kmB,EURO5,Non,145500,NaN,NaN,NaN,NaN,NaN,NaN
14,Volkswagen,E106053751,VOLKSWAGEN GOLF 1.6 TDI 115CH BI-TON GPS JOIN ...,GOLF 7,20450,2019,non requis,Diesel,mécanique,gris,...,106 g/kmB,NaN,Non,19500,gris,48 mois,NaN,NaN,NaN,NaN
16,Volkswagen,E106101572,VOLKSWAGEN GOLF VII (2) 1.6 TDI 115 BLUEMOTION...,GOLF 7,20300,2018,non requis,Diesel,automatique,gris clair,...,102 g/kmB,EURO6,Non,8627,NaN,12 mois,NaN,en cours,NaN,NaN


In [103]:
filtered_price = filtered_modele.loc[filtered_modele.Prix>int(min_price)].loc[filtered_modele.Prix<int(max_price)]

In [104]:
filtered_price

,Marque,Référence annonce,Nom,Modèle,Prix,Année,Contrôle technique,Énergie,Boîte de vitesse,Couleur extérieure,...,Émissions de CO2,Norme Euro,Éligible prime à la conversion,Kilométrage compteur km,Couleur intérieure,Garantie,Nombre de propriétaires,Garantie constructeur,Vérifié & Garanti,Provenance
4,Volkswagen,W101713798,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,12900,2015,non requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO6,Non,94000,tissu noir,6 mois,2,NaN,NaN,NaN
6,Volkswagen,W101713398,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10900,2013,requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO5,Non,134000,tissu noir,6 mois,3,NaN,NaN,NaN
8,Volkswagen,W101691329,VOLKSWAGEN GOLF VII 1.6 TDI 110 BLUEMOTION TEC...,GOLF 7,13490,2016,non requis,Diesel,mécanique,noir metal,...,99 g/kmA,EURO6,Non,75000,tissu gris,6 mois,1,NaN,NaN,NaN
13,Volkswagen,E106324636,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10990,2013,requis,Diesel,automatique,gris,...,102 g/kmB,EURO5,Non,145500,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
sorted_price = filtered_price.sort_values('Prix',ascending=True)

In [106]:
sorted_price

,Marque,Référence annonce,Nom,Modèle,Prix,Année,Contrôle technique,Énergie,Boîte de vitesse,Couleur extérieure,...,Émissions de CO2,Norme Euro,Éligible prime à la conversion,Kilométrage compteur km,Couleur intérieure,Garantie,Nombre de propriétaires,Garantie constructeur,Vérifié & Garanti,Provenance
6,Volkswagen,W101713398,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10900,2013,requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO5,Non,134000,tissu noir,6 mois,3,NaN,NaN,NaN
13,Volkswagen,E106324636,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10990,2013,requis,Diesel,automatique,gris,...,102 g/kmB,EURO5,Non,145500,NaN,NaN,NaN,NaN,NaN,NaN
4,Volkswagen,W101713798,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,12900,2015,non requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO6,Non,94000,tissu noir,6 mois,2,NaN,NaN,NaN
8,Volkswagen,W101691329,VOLKSWAGEN GOLF VII 1.6 TDI 110 BLUEMOTION TEC...,GOLF 7,13490,2016,non requis,Diesel,mécanique,noir metal,...,99 g/kmA,EURO6,Non,75000,tissu gris,6 mois,1,NaN,NaN,NaN


In [107]:
sorted_year = sorted_price.sort_values('Année',ascending=False)

In [108]:
sorted_year

,Marque,Référence annonce,Nom,Modèle,Prix,Année,Contrôle technique,Énergie,Boîte de vitesse,Couleur extérieure,...,Émissions de CO2,Norme Euro,Éligible prime à la conversion,Kilométrage compteur km,Couleur intérieure,Garantie,Nombre de propriétaires,Garantie constructeur,Vérifié & Garanti,Provenance
8,Volkswagen,W101691329,VOLKSWAGEN GOLF VII 1.6 TDI 110 BLUEMOTION TEC...,GOLF 7,13490,2016,non requis,Diesel,mécanique,noir metal,...,99 g/kmA,EURO6,Non,75000,tissu gris,6 mois,1,NaN,NaN,NaN
4,Volkswagen,W101713798,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,12900,2015,non requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO6,Non,94000,tissu noir,6 mois,2,NaN,NaN,NaN
6,Volkswagen,W101713398,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10900,2013,requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO5,Non,134000,tissu noir,6 mois,3,NaN,NaN,NaN
13,Volkswagen,E106324636,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10990,2013,requis,Diesel,automatique,gris,...,102 g/kmB,EURO5,Non,145500,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
sorted_kilometrage = sorted_year.sort_values('Kilométrage compteur km',ascending=True)

In [110]:
sorted_kilometrage

,Marque,Référence annonce,Nom,Modèle,Prix,Année,Contrôle technique,Énergie,Boîte de vitesse,Couleur extérieure,...,Émissions de CO2,Norme Euro,Éligible prime à la conversion,Kilométrage compteur km,Couleur intérieure,Garantie,Nombre de propriétaires,Garantie constructeur,Vérifié & Garanti,Provenance
8,Volkswagen,W101691329,VOLKSWAGEN GOLF VII 1.6 TDI 110 BLUEMOTION TEC...,GOLF 7,13490,2016,non requis,Diesel,mécanique,noir metal,...,99 g/kmA,EURO6,Non,75000,tissu gris,6 mois,1,NaN,NaN,NaN
4,Volkswagen,W101713798,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,12900,2015,non requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO6,Non,94000,tissu noir,6 mois,2,NaN,NaN,NaN
6,Volkswagen,W101713398,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10900,2013,requis,Diesel,mécanique,blanc verni,...,99 g/kmA,EURO5,Non,134000,tissu noir,6 mois,3,NaN,NaN,NaN
13,Volkswagen,E106324636,VOLKSWAGEN GOLF VII 1.6 TDI 105 BLUEMOTION TEC...,GOLF 7,10990,2013,requis,Diesel,automatique,gris,...,102 g/kmB,EURO5,Non,145500,NaN,NaN,NaN,NaN,NaN,NaN
